In [1]:
!pip install mvnx
import numpy as np
import mvnx
from scipy.signal import butter, filtfilt

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
class LoadMvnx:
    def __init__(self, filename):
        self.data = mvnx.load(filename)
        self.frameRate = 60
        self.segments = list(self.data.segments.values())
        self.raw_tc = list(self.data.timecode)
        self.pc_tc = list(self.data.ms)
        self.overall_output = {}
        self.run()

    def run(self):
        for i in range(len(self.raw_tc)):
            output = {}
            output['smpte_tc']=self.raw_tc[i]
            output['pc_tc']=self.pc_tc[i]
            output['orientation'] = self.get_seg_info(i,'orientation')
            output['position'] = self.get_seg_info(i,'position')
            output['velocity'] = self.get_seg_info(i,'velocity')
            output['acceleration'] = self.get_seg_info(i,'acceleration')
            output['angular acceleration'] = self.get_seg_info(i,'angular acceleration')
            output['angular velocity'] = self.get_seg_info(i,'angular velocity')
            self.overall_output['frame_'+str(i)] = output

    def filter(self,data):
        Fc = 6  # Desired effective cutoff frequency. (Winter, 2009)
        p = 2  # Number of filter passes
        Wc = Fc / np.power((2 ** (1 / p)) - 1,
                           1 / 4)  # Actual cutoff frequency to be applied after accounting for number of passes (Winter, 2009)
        Wn = Wc / (self.frameRate / 2)  # Cutoff frequency over half the original frequency
        b, a = butter(2, Wn, 'low')  # Butterworth filter coefficients
        return filtfilt(b, a, data)  # Applying Butterworth filter on acceleration

    def get_seg_info(self,frame,info):
        if info == 'orientation': data = self.data.orientation
        elif info == 'position': data = self.data.position
        elif info == 'velocity': data = self.data.velocity
        elif info == 'acceleration': data = self.data.acceleration
        elif info == 'angular velocity': data = self.data.angularVelocity
        elif info == 'angular acceleration': data = self.data.angularAcceleration
        else: return None
        info_dict = {}
        #data = self.filter(data)
        if info == 'orientation': info_arr = np.reshape(data, (len(data), 23, 4))
        else: info_arr = np.reshape(data, (len(data), 23, 3))
        index = 0
        for segment in self.segments:
            info_dict[segment] = list(info_arr[frame,index,:])
            index+=1
        return info_dict

    def extract_info(self):
        return self.overall_output, len(self.overall_output)

    def extract_segment_names(self):
        return self.segments

In [5]:
loader = LoadMvnx("/Users/asheshlalshrestha/Desktop/Datanal/EEGProject/lightlift-5.mvnx")
output, length = loader.extract_info()
print('number of frame:',length)
print('segment names:',loader.extract_segment_names())
print('start_position_righthand:',output['frame_0']['position']['RightHand'])
print('last_position_righthand:',output['frame_'+str(length-1)]['position']['RightHand'])

number of frame: 144
segment names: ['Pelvis', 'L5', 'L3', 'T12', 'T8', 'Neck', 'Head', 'RightShoulder', 'RightUpperArm', 'RightForeArm', 'RightHand', 'LeftShoulder', 'LeftUpperArm', 'LeftForeArm', 'LeftHand', 'RightUpperLeg', 'RightLowerLeg', 'RightFoot', 'RightToe', 'LeftUpperLeg', 'LeftLowerLeg', 'LeftFoot', 'LeftToe']
start_position_righthand: [2.784659, 1.315917, 0.393573]
last_position_righthand: [2.470741, 1.786028, 1.143362]
